## Preprocessament

In [1]:
import pandas as pd
import numpy as np

C:\Users\quimm\AppData\Local\Temp\ipykernel_8168\2162656668.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


Lectura de dades

In [2]:
activitats_v = pd.read_csv("data/raw/activitats_valencia.csv")
activitats_a = pd.read_csv("data/raw/activitats_alacant.csv")
activitats_c = pd.read_csv("data/raw/activitats_castello.csv")
ajuntaments_info = pd.read_csv("data/raw/ajuntaments_info_raw.csv")

In [3]:
activitats_v.columns

Index(['Marca de temps', 'Localitat', 'Data', 'Categoria',
       'En cas d'altres, quina?', 'Nom de l'activitat ', 'Companyia, artista',
       'Lloc', 'Llengua de l'activitat', 'Preu', 'Regidoria organitzadora',
       'Llengua de la publicitat', 'Font',
       'Persona que ha introduït les dades', 'Unnamed: 14', 'Unnamed: 15',
       'Estructura dades', 'Observacions estructura dades'],
      dtype='object')

In [4]:
ajuntaments_info.columns

Index(['MUNICIPI', 'PROVÍNCIA', 'COMARCA', 'N HABITANTS',
       'ÌNDEX ENVELLIMENT POBLACIÓ (2021)', 'POBLACIÓ TURÍSTICA',
       'PARTIT GOVERNANT EN 2022-23', 'PARTIT GOVERNANT 2023-24',
       'Subvenció normalització\n de l'us del valencià \nper part de la Generalitat (2022)',
       'LLENGUA PREDOMINANT', 'Regidoria Cultura 2022',
       'Regidoria Cultura 2023'],
      dtype='object')

* Seleccio de variables i canvi de nom

In [96]:
columns = ["Marca temporal", "Localitat", "Data", "Categoria", "En cas d'altres, quina?", "Nom de l'activitat ", "Companyia, artista", "Lloc", "Llengua de l'activitat", "Preu", "Regidoria organitzadora", "Llengua de la publicitat", "Font",  "Persona que ha introduït les dades"]
columns_v = ["Marca de temps", "Localitat", "Data", "Categoria", "En cas d'altres, quina?", "Nom de l'activitat ", "Companyia, artista", "Lloc", "Llengua de l'activitat", "Preu", "Regidoria organitzadora", "Llengua de la publicitat", "Font",  "Persona que ha introduït les dades"]
column_rename = ["marca_temps", "localitat", "data", "categoria", "categoria_alternativa", "nom_activitat", "companyia", "lloc", "llengua_activitat", "preu", "regidoria", "llengua_publicitat", "font", "persona_dades"]
columns_info_rename = ["ajuntament", "provincia", "comarca", "n_habitants", "index_envelliment_2021", "pob_turistica", "govern_22_23", "govern_23_24",  "subv_us_valencia_2022", "predomini_ling", "regidoria_cultura", "regidoria_cultura_2023"]

In [97]:
activitats_v = activitats_v[columns_v]
activitats_v.columns = column_rename
activitats_c = activitats_c[columns]
activitats_c.columns = column_rename
activitats_a = activitats_a[columns]
activitats_a.columns = column_rename
ajuntaments_info.columns = columns_info_rename

* Juntem els 3 datasets per a conformar el dataset activitats

In [98]:
activitats = pd.concat([activitats_a, activitats_v, activitats_c], axis = 0)

In [99]:
activitats.shape

(8396, 14)

## Preprocessament de dades

* Tranformacions en variables

**ajuntaments_info**

pob_turistica

In [100]:
# Adegim el valor "No" a aquelles poblacions que falten dades
ajuntaments_info["pob_turistica"].loc[ajuntaments_info["pob_turistica"].isnull()] = "No"

C:\Users\quimm\AppData\Local\Temp\ipykernel_12496\1527247745.py:2: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  ajuntaments_info["pob_turistica"].loc[ajuntaments_info["pob_turistica"].isnull()] = "No"
C:\Users\quimm\AppData\Local\Temp\ipyke

index_envelliment_2021

In [101]:
ajuntaments_info["index_envelliment_2021"] = pd.to_numeric(ajuntaments_info["index_envelliment_2021"].str.replace(",", ".") )

subv_us_valencia_2022

In [102]:
ajuntaments_info["subv_us_valencia_2022"] = pd.to_numeric(ajuntaments_info["subv_us_valencia_2022"].str.replace(".", "").str.replace("€", "").str.replace(",", ".") )

**activitats**

llengua_activitat

In [103]:
def assign_value_more_than_2_languages(llengua_activitat):
    """
    si la variable té dos llengues o més, el valor canvia a "Dos o més llengües"
    """
    llengua_activitat_split = llengua_activitat.split(", ")

    if len(llengua_activitat_split) > 1:
        if "Activitat sense llengua" not in llengua_activitat_split and "No hi ha informació" not in llengua_activitat_split:
            return "Dos o més llengües"
    
    return llengua_activitat

In [104]:
activitats["llengua_activitat"] = activitats["llengua_activitat"].apply(lambda x: assign_value_more_than_2_languages(x))

localitat

In [105]:
activitats = activitats.loc[activitats["localitat"] != "Bilingüe"]
activitats = activitats.loc[activitats["localitat"] != "Monforte del Cid"]

data

In [106]:
activitats = activitats.loc[~pd.to_datetime(activitats.data, format= "%d/%m/%Y").dt.month.isin([4,5])]

Variables de text

In [107]:
activitats.nom_activitat=activitats.nom_activitat.str.replace(",", ";")
activitats.nom_activitat = activitats.nom_activitat.str.replace("\n", "")
activitats.nom_activitat = activitats.nom_activitat.str.replace("\r", "")
activitats.companyia = activitats.companyia.str.replace(",", ";")
activitats.lloc = activitats.lloc.str.replace(",", ";")
activitats.preu = activitats.preu.str.replace(",", ";")

* Creació de noves variables

Preu_mitjà

In [108]:
activitats["preu_mitja"] = activitats.preu.str.findall(r"\d+[,'.]?\d*")
activitats["preu_mitja"].loc[activitats.preu.notnull()] = activitats.loc[activitats.preu.notnull()].preu_mitja.apply(lambda x: np.float64(np.mean([float(num.replace("'", ".").replace(",", ".")) for num in x])))

C:\Users\quimm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\quimm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
C:\Users\quimm\AppData\Local\Temp\ipykernel_12496\3214746594.py:2: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical

In [109]:
type(activitats["preu_mitja"].loc[activitats.preu.notnull()][0])

float

In [110]:
# Modifiquem el . per , per problemes de codificacio en els analistes
#activitats["preu_mitja"] = activitats["preu_mitja"].apply(lambda num: str(num).replace(".", ","))

Categoria New

In [111]:
def assign_category(category_alternative):

    if type(category_alternative) != str:
        category_new = np.nan
    else:
        category_alternative = category_alternative.lower()
        if "fall" in category_alternative:
            category_new = "Falles"
        elif "màg" in category_alternative:
            category_new = "Màgia"
        elif "visita" in category_alternative:
            category_new = "Activitats sobre patrimoni"
        elif "circ" in category_alternative:
            category_new = "Circ"
        elif "concu" in category_alternative:
            category_new = "Concurs"
        elif "premi" in category_alternative:
            category_new = "Premis"
        elif "taller" in category_alternative:
            category_new = "Taller"
        elif "espectac" in category_alternative:
            category_new = "Espectacle"
        else:
            category_new = np.nan

    return category_new


In [112]:
activitats["categoria_new"] = activitats["categoria_alternativa"].apply(assign_category)
activitats["categoria_new"] = np.where(activitats['categoria_new'].isnull(), activitats['categoria'], activitats['categoria_new'])

Mes

In [113]:
activitats["Mes"] = pd.to_datetime(activitats.data, format= "%d/%m/%Y").dt.month_name()

Any de l'analisi

In [114]:
def set_analisi_year(date_):
    if date_ < pd.to_datetime("2023/06/30"):
        res = "analisi_2023"
    elif date_ < pd.to_datetime("2025/06/30"):
        res = "analisi_2024"

    return res

In [115]:
pd.to_datetime(activitats.data, format="%d/%m/%Y").describe()

count                             8373
mean     2023-01-29 06:33:50.240057344
min                2022-10-01 00:00:00
25%                2022-11-11 00:00:00
50%                2022-12-18 00:00:00
75%                2023-02-24 00:00:00
max                2024-12-21 00:00:00
Name: data, dtype: object

In [116]:
activitats.loc[pd.to_datetime(activitats.data, format="%d/%m/%Y") > pd.to_datetime("2024/06/30")]

,marca_temps,localitat,data,categoria,categoria_alternativa,nom_activitat,companyia,lloc,llengua_activitat,preu,regidoria,llengua_publicitat,font,persona_dades,preu_mitja,categoria_new,Mes
2592,13/02/2024 13:16:39,"Puig de Santa Maria, el",21/12/2024,Teatre,NaN,Clic!,Julio Martí Zahonero,NaN,No hi ha informació,3 euros,Cultura,Bilingüe,https://www.facebook.com/ajuntament.delpuig.1/...,Irene,3.0,Teatre,December
4137,09/02/2024 9:53:17,València,16/12/2024,Teatre,NaN,La voluntad de creer,Pablo Messiez,Teatre El Musical,Espanyol,12 euros,Cultura,Espanyol,https://www.valencia.es/web/guest/w/la-volunta...,Irene,12.0,Teatre,December


In [117]:
activitats["any_estudi"] = pd.to_datetime(activitats.data, format="%d/%m/%Y").apply(lambda x: set_analisi_year(x))

* Nou conjunt de dades: activitats_summaries.csv

llengua activitat

In [118]:
grup1 = activitats.groupby(["localitat","llengua_activitat"])["llengua_activitat"].count()
grup1 = pd.DataFrame(grup1)
grup1.columns = ["count_llengua"]
grup1= grup1.reset_index()

grup1 = grup1.groupby("localitat").apply(lambda x : x.sort_values(by = 'count_llengua', ascending = False).head(3).reset_index(drop = True))
grup1.columns = ["localitat_col", 'llengua_activitat', 'count_llengua']

llengua_activitat_top1 = grup1.groupby("localitat").apply(lambda x : x.head(1).reset_index(drop = True))["llengua_activitat"]
llengua_activitat_top2 = grup1.groupby("localitat").apply(lambda x : x.head(2).tail(1).reset_index(drop = True))["llengua_activitat"]
llengua_activitat_top3 = grup1.groupby("localitat").apply(lambda x : x.tail(1).reset_index(drop = True))["llengua_activitat"]

C:\Users\quimm\AppData\Local\Temp\ipykernel_12496\334360915.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grup1 = grup1.groupby("localitat").apply(lambda x : x.sort_values(by = 'count_llengua', ascending = False).head(3).reset_index(drop = True))


llengua publicitat

In [119]:
grup1 = activitats.groupby(["localitat","llengua_publicitat"])["llengua_publicitat"].count()
grup1 = pd.DataFrame(grup1)
grup1.columns = ["count_llengua"]
grup1= grup1.reset_index()

grup1 = grup1.groupby("localitat").apply(lambda x : x.sort_values(by = 'count_llengua', ascending = False).head(2).reset_index(drop = True))
grup1.columns = ["localitat_col", 'llengua_activitat', 'count_llengua']

llengua_publicitat_top1 = grup1.groupby("localitat").apply(lambda x : x.head(1).reset_index(drop = True))["llengua_activitat"]
llengua_publicitat_top2 = grup1.groupby("localitat").apply(lambda x : x.tail(1).reset_index(drop = True))["llengua_activitat"]

C:\Users\quimm\AppData\Local\Temp\ipykernel_12496\3631808114.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grup1 = grup1.groupby("localitat").apply(lambda x : x.sort_values(by = 'count_llengua', ascending = False).head(2).reset_index(drop = True))


Language position

In [120]:
grup1 = activitats.groupby(["localitat","llengua_activitat"])["llengua_activitat"].count()
grup1 = pd.DataFrame(grup1)
grup1.columns = ["count_llengua"]
grup1= grup1.reset_index()

grup1 = grup1.groupby("localitat").apply(lambda x : x.sort_values(by = 'count_llengua', ascending = False).reset_index(drop = True))
grup1.columns = ["localitat_col", 'llengua_activitat', 'count_llengua']

C:\Users\quimm\AppData\Local\Temp\ipykernel_12496\2141191287.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grup1 = grup1.groupby("localitat").apply(lambda x : x.sort_values(by = 'count_llengua', ascending = False).reset_index(drop = True))


In [121]:
pos_val = grup1.groupby("localitat").apply(lambda x : x.index[x["llengua_activitat"]=="Valencià/català"])
pos_val = pd.DataFrame(pos_val)

pos_cas = grup1.groupby("localitat").apply(lambda x : x.index[x["llengua_activitat"]=="Espanyol"])
pos_cas = pd.DataFrame(pos_cas)

In [122]:
def get_position(array):
    "get the position of the language in the most used ones. If not used, number 5 assigned"
    try:
        return array[0][0][1]
    except:
        return 5

In [123]:
positions_val = []

for i in range(pos_val.shape[0]):
    positions_val.append(get_position(pos_val.iloc[i]))

In [124]:
positions_cas = []

for i in range(pos_cas.shape[0]):
    positions_cas.append(get_position(pos_cas.iloc[i]))

N activitats

In [125]:
grup1 = activitats.groupby(["localitat"])["llengua_activitat"].count()
n_activivitats = grup1.values

* Creating datasets

activitats_summaries

In [126]:
activitats_summaries = pd.DataFrame(llengua_activitat_top1)
activitats_summaries = activitats_summaries.reset_index()
del activitats_summaries["level_1"]
activitats_summaries.columns = ['localitat', 'llengua_activitat_top1']

In [127]:
activitats_summaries['llengua_activitat_top2'] = llengua_activitat_top2.values
activitats_summaries['llengua_activitat_top3'] = llengua_activitat_top3.values
activitats_summaries['llengua_publicitat_top1'] = llengua_publicitat_top1.values
activitats_summaries['llengua_publicitat_top2'] = llengua_publicitat_top2.values
activitats_summaries["position_val_in_llengua_activitat"] = positions_val
activitats_summaries["position_esp_in_llengua_activitat"] = positions_cas
activitats_summaries["n_activitats"] = n_activivitats

In [128]:
#ratio_activitats_nhabitants
aux = activitats_summaries.merge(ajuntaments_info[["ajuntament", "n_habitants"]], how ='left', left_on="localitat", right_on = "ajuntament")
activitats_summaries["ratio_activitats_nhabitants"] = aux["n_activitats"]/(aux["n_habitants"]/1000)

Saving Datasets

In [129]:
activitats.to_csv("data/activitats.csv", index=False)

In [130]:
activitats_summaries.to_csv("data/activitats_summaries.csv", index=False)

In [131]:
ajuntaments_info.to_csv("data/ajuntaments_info.csv", index=False)